# Displaying text data in TensorBoard

## Overview

使用 TensorFlow  Text Summary API，您可以轻松记录任意文本并在 TensorBoard中查看。这对于取样和检查您的输入数据，或记录执行元数据或生成的文本非常有帮助。您还可以将诊断数据记录为文本，这有助于您的模型开发过程。

在本教程中，您将尝试 Text Summary API 的一些基本用例。

## Setup

In [1]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [2]:
import tensorflow as tf

from datetime import datetime
import json
from packaging import version
import tempfile

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.6.0


## Logging a single piece of text

要了解 Text Summary API的工作原理，您只需记录一些文本，并查看它在 TensorBoard 中的显示方式。

In [3]:
my_text = "Hello world! 😃"

In [4]:
# Clear out any prior log data.
!rm -rf logs

logdir = "logs/text_basics/" + datetime.now().strftime("%Y%m%d-%H%M%S")

file_writer = tf.summary.create_file_writer(logdir)

with file_writer.as_default():
    tf.summary.text("first_text", my_text, step=0)

Metal device set to: Apple M1


2022-05-09 09:53:43.275237: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-09 09:53:43.275376: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


现在，使用TensorBoard检查文本。等待几秒钟。

In [5]:
%tensorboard --logdir logs

## Organizing multiple text streams

如果您有多个文本流，您可以将它们保存在单独的命名空间中，以帮助组织它们，就像标量或其他数据一样。

请注意，如果您在许多步骤中记录文本，TensorBoard将对要显示的步骤进行子采样，以使演示易于管理。您可以使用 `--samples_per_plugin` 标志控制采样率。

In [6]:
logdir = "logs/multiple_texts/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir)

with file_writer.as_default():
    with tf.name_scope("name_scope_1"):
        for step in range(20):
            tf.summary.text("a_stream_of_text", f"Hello from step {step}", step=step)
            tf.summary.text("another_stream_of_text", f"This can be kept separate {step}", step=step)
    with tf.name_scope("name_scope_2"):
        tf.summary.text("just_from_step_0", "This is an important announcement from step 0", step=0)

In [7]:
%tensorboard --logdir logs/multiple_texts --samples_per_plugin 'text=5'

## Markdown interpretation

TensorBoard 将 Text Summary 解释为Markdown，因为 rich formatting 可以使您记录的数据更容易阅读和理解，如下所示。

In [10]:
logdir = "logs/markdown/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir)

some_obj_worth_nothing = {
    "tfds_training_data": {
        "name": "mnist",
        "split": "train",
        "shuffle_files": "True",
    },
    "keras_optimizer": {
        "name": "Adagrad",
        "learning_rate": "0.001",
        "epsilon": 1e-07,
    },
    "hardware": "Cloud TPU",
}

def pretty_json(hp):
    json_hp = json.dumps(hp, indent=2)
    return "".join("\t" + line for line in json_hp.splitlines(True))

markdown_text = """
### Markdown Text

TensorBoard supports basic markdown syntax, including:

    preformatted code

**bold text**

| and | tables |
| ---- | ---------- |
| among | others |
"""

with file_writer.as_default():
    tf.summary.text("run_params", pretty_json(some_obj_worth_nothing), step=0)
    tf.summary.text("markdown_jubiliee", markdown_text, step=0)

In [11]:
%tensorboard --logdir logs/markdown